In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])

In [61]:
## change ##
batch_size = 16
clip = .5
predict_len = 100

Prepare data

In [62]:
train_gen = SequenceGenLM(train,predict_len=predict_len)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenLM(train,False,predict_len=predict_len)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Pre-train on LM

In [63]:
convNets = Sequential(ConvBatchLeaky1D(1,2,5,stride=2),
                      ConvBatchLeaky1D(2,4,5,stride=2),
                      ConvBatchLeaky1D(4,8,5,stride=2),
                      ConvBatchLeaky1D(8,16,5,stride=2),
                      ConvBatchLeaky1D(16,32,5,stride=2),
                      DenseBlock(32),
                      DenseBlock(64),
                      Dropout(0.5),
                      DenseBlock(128),
                      Dropout(0.5),
                      DenseBlock(256),
                      Dropout(0.5),
                      GRU_NCL(512,512,batch_first=True,bidirectional=True,returnH=True))

In [64]:
linear = Sequential(Dropout(0.5),Linear(512*2,512),LeakyReLU(0.1,True),Dropout(0.33),Linear(512,predict_len))
model = CNN_RNN2seq(convNets,linear).to('cuda:0')
opt = Adam(trainable_parameter(model),lr=1e-3)
loss_func_nor = loss_func_generator(L1Loss())

In [65]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:0.33779362159279674, val_loss:0.32968881726264954
epoch:1, train_loss:0.3333536077004213, val_loss:0.3297840356826782
epoch:2, train_loss:0.3327339431987359, val_loss:0.32967787981033325
epoch:3, train_loss:0.33227915001603275, val_loss:0.3303286135196686
epoch:4, train_loss:0.33220566938129753, val_loss:0.3296218812465668
Training completed in 209.35570120811462s


Fine tune head model

In [66]:
train_gen2 = SequenceGen(train)
train_gen2 = DataLoader(train_gen2,batch_size,False,num_workers=2)

val_gen2 = SequenceGen(train,False)
val_gen2 = DataLoader(val_gen2,batch_size,False,num_workers=2)

In [67]:
set_requires_grad(convNets, False)
linear = Sequential(Dropout(0.5),Linear(512*2,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))
model = CNN_RNN2seq(convNets,linear).to('cuda:0')
opt = Adam(trainable_parameter(model),lr=1e-3)

In [68]:
model = fit(5, model, loss_func_nor, opt, train_gen2, val_gen2,clip=clip)

epoch:0, train_loss:3.236539992575462, val_loss:3.1511504650115967
epoch:1, train_loss:2.8423485503746915, val_loss:3.057168483734131
epoch:2, train_loss:2.7921510906173634, val_loss:3.0570342540740967
epoch:3, train_loss:2.791817692609934, val_loss:3.0051629543304443
epoch:4, train_loss:2.766806823702959, val_loss:3.015470266342163
Training completed in 125.04541254043579s


Fine tune all layers

In [69]:
set_requires_grad(convNets, True)
opt = Adam(differential_lr(convNets,2e-4,factor=2),lr=4e-4)

In [70]:
model = fit(5, model, loss_func_nor, opt, train_gen2, val_gen2,clip=clip)

epoch:0, train_loss:2.7176562839975724, val_loss:2.96711802482605
epoch:1, train_loss:2.6125742374704433, val_loss:3.1068906784057617
epoch:2, train_loss:2.543830138559525, val_loss:2.853834629058838
epoch:3, train_loss:2.435639671408213, val_loss:3.001256227493286
epoch:4, train_loss:2.406792877958371, val_loss:2.866123676300049
Training completed in 258.2264096736908s


Submission

In [102]:
submission = pd.read_csv('../Data/sample_submission.csv',)

In [145]:
test_gen = SequenceGenTest(submission.seg_id.tolist())
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [155]:
submission.iloc[:,1] = predict(model,test_gen)

In [164]:
submission.to_csv('../Submission/sub1.csv',index=False)